<a href="https://colab.research.google.com/github/omkarpat/EmpatheticDialoguesEmotionDetection/blob/master/Restrictive_Collaping_Emotion_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 245\ Project
#check if drive is loaded successfully
!ls Data

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/CSE 245 Project
'Data Exploration.ipynb'
 data_fixed_train_complement_analysis.csv
 data_fixed_train.json
 data_sample_100.json
 data_sample_10_complement_analysis.csv
 data_sample_10.json
 data_sample_10_processed.json
 data_sample_fixed_processed_final.json
 data_sample_fixed_processed_model1_final.csv
 data_sample_fixed_processed_model2_final.csv
 data_sample_fixed_processed_model3_final.csv
 data_sample_fixed_processed_model4_final.csv
 fixed
 fixed_test.csv
 fixed_test.json
 fixed_train_516.csv
 fixed_valid.csv
 fixed_valid.json
'informative words.ipynb'
 model1_predictions_test.csv
 model1_predictions_train.csv
 model1_predictions_valid.csv
 model2_predictions_test.csv
 model2_predictions_train.csv
 model2_predictions_valid.csv
 model3_predictions_test.csv
 model3_predictions_train.csv
 model3_predictions_valid.csv
 model4_pre

In [2]:
# install supporting libraries
!pip install transformers

In [3]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [4]:
dataset_name = "model4"
model_name = "model4"
model_type = "more_restrictive_collapsed_emotion" #pos_neg or emotion or valence or collapsed_emotion or more_restrictive_collapsed_emotion/less_restrictive_collapsed_emotion

In [5]:
import pandas as pd
import numpy as np
train_filename = "Data/data_sample_fixed_processed_" + dataset_name + "_final.csv"
valid_filename = "Data/valid_fixed_processed_" + dataset_name + "_final.csv"
test_filename = "Data/test_fixed_processed_" + dataset_name + "_final.csv"

train_df = pd.read_csv(train_filename, sep='|', error_bad_lines=False)
train_df = train_df.dropna()
valid_df = pd.read_csv(valid_filename, sep='|', error_bad_lines=False)
valid_df = valid_df.dropna()
test_df = pd.read_csv(test_filename, sep='|', error_bad_lines=False)
test_df = test_df.dropna()

In [6]:
category2index_more_restrictive_model2 = {
    "guilty":0,
    "ashamed":0,
    "furious":1,
    "angry":1,
    "terrified":2,
    "afraid":2,
    "sentimental":3,
    "sad":4,
    "hopeful":5,
    "joyful":6,
    "impressed":7,
    "proud":8,
    "grateful":9,
    "lonely":10,
    "surprised":11,
    "anxious":12,
    "trusting":13,
    "nostalgic":14,
    "excited":15,
    "disgusted":16,
    "caring":17,
    "prepared":18,
    "disappointed":19,
    "faithful":20,
    "confident":21,
    "apprehensive":22,
    "jealous":23,
    "embarrassed":24,
    "devastated":25,
    "annoyed":26,
    "content":27,
    "anticipating":28
}

category2index_more_restrictive_model4 = {
    "guilty":0,
    "ashamed":0,
    "furious":1,
    "angry":1,
    "terrified":2,
    "afraid":2,
    "nostalgic":3,
    "sentimental":3,
    "sad":4,
    "hopeful":5,
    "joyful":6,
    "impressed":7,
    "proud":8,
    "grateful":9,
    "lonely":10,
    "surprised":11,
    "anxious":12,
    "trusting":13,
    "excited":14,
    "disgusted":15,
    "caring":16,
    "prepared":17,
    "disappointed":18,
    "faithful":19,
    "confident":20,
    "apprehensive":21,
    "jealous":22,
    "embarrassed":23,
    "devastated":24,
    "annoyed":25,
    "content":26,
    "anticipating":27
}

category2index_less_restrictive_model2 = {
    "guilty":0,
    "ashamed":0,
    "furious":1,
    "angry":1,
    "terrified":2,
    "afraid":2,
    "nostalgic":3,
    "sentimental":3,
    "anxious":4,
    "apprehensive":4,
    "excited":5,
    "anticipating":5,
    "sad":6,
    "hopeful":7,
    "joyful":8,
    "impressed":9,
    "proud":10,
    "grateful":11,
    "lonely":12,
    "surprised":13,
    "trusting":14,
    "disgusted":15,
    "caring":16,
    "prepared":17,
    "disappointed":18,
    "faithful":19,
    "confident":20,
    "jealous":21,
    "embarrassed":22,
    "devastated":23,
    "annoyed":24,
    "content":25    
}

category2index_less_restrictive_model4 = {
    "guilty":0,
    "ashamed":0,
    "furious":1,
    "angry":1,
    "terrified":2,
    "afraid":2,
    "nostalgic":3,
    "sentimental":3,
    "anxious":4,
    "apprehensive":4,
    "excited":5,
    "joyful":5,
    "anticipating":6,
    "sad":7,
    "hopeful":8,
    "impressed":9,
    "proud":10,
    "grateful":11,
    "lonely":12,
    "surprised":13,
    "trusting":14,
    "disgusted":15,
    "caring":16,
    "prepared":17,
    "disappointed":18,
    "faithful":19,
    "confident":20,
    "jealous":21,
    "embarrassed":22,
    "devastated":23,
    "annoyed":24,
    "content":25    
}

category2index = {
    "nostalgic":1,
    "sentimental":1,
    "sad":2,
    "devastated":2,
    "hopeful":3,
    "joyful":4,
    "impressed":5,
    "terrified":6,
    "afraid":6,
    "proud":7,
    "grateful":8,
    "lonely":9,
    "surprised":10,
    "anxious":11,
    "guilty":12,
    "trusting":13,
    "excited":15,
    "disgusted":16,
    "caring":17,
    "furious":18,
    "angry":18,
    "prepared":19,
    "disappointed":20,
    "faithful":21,
    "confident":22,
    "apprehensive":23,
    "jealous":24,
    "embarrassed":25,
    "annoyed":26,
    "ashamed":27,
    "content":28,
    "anticipating":29
}
if model_name == "model2":
  if model_type == "more_restrictive_collapsed_emotion":
    category2index = category2index_more_restrictive_model2
  else:
    category2index = category2index_less_restrictive_model2
if model_name == "model4":
  if model_type == "more_restrictive_collapsed_emotion":
    category2index = category2index_more_restrictive_model4
  else:
    category2index = category2index_less_restrictive_model4

number_of_labels = len(set(category2index.values()))
#print(category2index)
print("Preparing data for training %s, with %s. Total Classes: %s" % (model_name, model_type, number_of_labels))
index2category = {}
for key, value in category2index.items():
  index2category[value] = key

def category2label(category):
  for key in category2index.keys():
    if key == category:
      return category2index[key]
  return 0

def label2emotion(labels):
  emotion_list = []
  for label in labels:
    emotion_list.append(index2category[label])
  return emotion_list

Preparing data for training model4, with more_restrictive_collapsed_emotion. Total Classes: 28


In [7]:
print('Number of training sentences: {:,}\n'.format(train_df.shape[0]))
print('Number of valid sentences: {:,}\n'.format(valid_df.shape[0]))
print('Number of test sentences: {:,}\n'.format(test_df.shape[0]))

train_df['label'] = train_df['emotion'].apply(lambda x: int(category2label(x.lower())))
valid_df['label'] = valid_df['emotion'].apply(lambda x: int(category2label(x.lower())))
test_df['label'] = test_df['emotion'].apply(lambda x: int(category2label(x.lower())))

# Display 10 random rows from the data.
train_df.sample(10)

Number of training sentences: 19,533

Number of valid sentences: 2,758

Number of test sentences: 2,538



,emotion,processed_emotion,text,vader_neg,vader_neu,vader_pos,vader_compound,textblob,flair_value,flair_confidence,label
8447,jealous,negative,My brother won the lottery last month. My brot...,0.087,0.659,0.254,0.9018,0.000000,NEGATIVE,0.999718,22
16611,surprised,positive,I had lost my iPad in an airport a week ago_co...,0.093,0.806,0.101,0.5376,0.002170,NEGATIVE,0.999997,11
1360,disgusted,negative,Why would anyone ever eat pees? Why would anyo...,0.056,0.672,0.272,0.8531,-0.218750,NEGATIVE,0.734531,15
14861,afraid,negative,Driving always scares me. I've seen some prett...,0.357,0.579,0.064,-0.9807,-0.342857,NEGATIVE,0.943693,2
1864,impressed,positive,I went to a soccer game the other night. the ...,0.000,0.716,0.284,0.9674,0.018029,POSITIVE,0.984071,7
792,joyful,positive,I recently went on vacation to Florida and it ...,0.000,0.723,0.277,0.9765,0.313636,POSITIVE,0.988497,6
17577,terrified,negative,I promised my sister that I would go sky divin...,0.129,0.778,0.093,-0.8055,-0.183333,NEGATIVE,0.999959,2
11820,apprehensive,negative,I hesitated before getting on the rollercoaste...,0.334,0.590,0.076,-0.9419,-0.083333,NEGATIVE,0.749355,21
9209,trusting,positive,I felt trusting of my uncle when he volunteere...,0.017,0.684,0.299,0.9874,0.655625,NEGATIVE,0.999897,13
18492,jealous,negative,Becky got the new promotion at work_comma_ I w...,0.113,0.744,0.143,0.4694,0.049411,NEGATIVE,0.999942,22


In [8]:
sentences_train = train_df.text.values
labels_train = train_df.label.values
sentences_valid = valid_df.text.values
labels_valid = valid_df.label.values
sentences_test = test_df.text.values
labels_test = test_df.label.values

In [9]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [10]:
# Print the original sentence.
print(' Original: ', sentences_train[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences_train[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences_train[0])))

 Original:  I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world. I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world. Was this a friend you were in love with_comma_ or just a best friend? This was a best friend. I miss her. Where has she gone? We no longer talk. Oh was this something that happened because of an argument?
Tokenized:  ['i', 'remember', 'going', 'to', 'the', 'fireworks', 'with', 'my', 'best', 'friend', '.', 'there', 'was', 'a', 'lot', 'of', 'people', '_', 'com', '##ma', '_', 'but', 'it', 'only', 'felt', 'like', 'us', 'in', 'the', 'world', '.', 'i', 'remember', 'going', 'to', 'see', 'the', 'fireworks', 'with', 'my', 'best', 'friend', '.', 'it', 'was', 'the', 'first', 'time', 'we', 'ever', 'spent', 'time', 'alone', 'together', '.', 'although', 'there', 'wa

In [11]:
longer_list = []
for index, sent in enumerate(sentences_train):
  if len(tokenizer.encode(sent, add_special_tokens=True)) > 500:
    longer_list.append(index)
sentences_train = np.delete(sentences_train, longer_list)
labels_train = np.delete(labels_train, longer_list)

max_len = 0
# For every sentence...
for sent in sentences_train:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max train sentence length: ', max_len)

max_len = 0
# For every sentence...
for sent in sentences_valid:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max valid sentence length: ', max_len)

max_len = 0
# For every sentence...
for sent in sentences_test:
    try:
      # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
      input_ids = tokenizer.encode(sent, add_special_tokens=True)
    except:
      print(sent)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max test sentence length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (554 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (542 > 512). Running this sequence through the model will result in indexing errors


Max train sentence length:  432
Max valid sentence length:  489
Max test sentence length:  442


In [12]:
def tokenize(sentences, labels):
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.

      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          truncation = 'only_first',
                          max_length = 512,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.from_numpy(labels)

  # Print sentence 0, now as a list of IDs.
  print('Original: ', sentences[0])
  print('Token IDs:', input_ids[0])
  return(input_ids, attention_masks, labels)

In [13]:
input_ids_train, attention_masks_train, labels_train = tokenize(sentences_train, labels_train)
input_ids_valid, attention_masks_valid, labels_valid = tokenize(sentences_valid, labels_valid)
input_ids_test, attention_masks_test, labels_test = tokenize(sentences_test, labels_test)

Original:  I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world. I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world. Was this a friend you were in love with_comma_ or just a best friend? This was a best friend. I miss her. Where has she gone? We no longer talk. Oh was this something that happened because of an argument?
Token IDs: tensor([  101,  1045,  3342,  2183,  2000,  1996, 16080,  2007,  2026,  2190,
         2767,  1012,  2045,  2001,  1037,  2843,  1997,  2111,  1035,  4012,
         2863,  1035,  2021,  2009,  2069,  2371,  2066,  2149,  1999,  1996,
         2088,  1012,  1045,  3342,  2183,  2000,  2156,  1996, 16080,  2007,
         2026,  2190,  2767,  1012,  2009,  2001,  1996,  2034,  2051,  2057,
         2412,  2985,  2051,  2894,  2362,  1012,  2348

In [14]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_valid = TensorDataset(input_ids_valid, attention_masks_valid, labels_valid)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset_train))
val_size = len(dataset_train) - train_size

# Divide the dataset by randomly selecting samples.
#train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

#Use the given training and validation datasets
train_dataset, val_dataset = dataset_train, dataset_valid

print('{:>5,} training samples'.format(len(train_dataset)))
print('{:>5,} validation samples'.format(len(val_dataset)))

19,530 training samples
2,758 validation samples


In [15]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [16]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = number_of_labels, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np
from transformers import get_linear_schedule_with_warmup

def train(dataloader, epochs=3):
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

  # Number of training epochs. The BERT authors recommend between 2 and 4. 
  # We chose to run for 4, but we'll see later that this may be over-fitting the
  # training data.
  #epochs = 15

  # Total number of training steps is [number of batches] x [number of epochs]. 
  # (Note that this is not the same as the number of training samples).
  total_steps = len(train_dataloader) * epochs

  # Create the learning rate scheduler.
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps)

  # Set the seed value all over the place to make this reproducible.
  seed_val = 42

  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  # We'll store a number of quantities such as training and validation loss, 
  # validation accuracy, and timings.
  training_stats = []

  # Measure the total training time for the whole run.
  total_t0 = time.time()

  # For each epoch...
  for epoch_i in range(0, epochs):
      
      # ========================================
      #               Training
      # ========================================
      
      # Perform one full pass over the training set.

      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      # Measure how long the training epoch takes.
      t0 = time.time()

      # Reset the total loss for this epoch.
      total_train_loss = 0

      # Put the model into training mode. Don't be mislead--the call to 
      # `train` just changes the *mode*, it doesn't *perform* the training.
      # `dropout` and `batchnorm` layers behave differently during training
      # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
      model.train()

      # For each batch of training data...
      for step, batch in enumerate(dataloader):

          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(dataloader), elapsed))

          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using the 
          # `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)

          # Always clear any previously calculated gradients before performing a
          # backward pass. PyTorch doesn't do this automatically because 
          # accumulating the gradients is "convenient while training RNNs". 
          # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
          model.zero_grad()        

          # Perform a forward pass (evaluate the model on this training batch).
          # The documentation for this `model` function is here: 
          # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
          # It returns different numbers of parameters depending on what arguments
          # arge given and what flags are set. For our useage here, it returns
          # the loss (because we provided labels) and the "logits"--the model
          # outputs prior to activation.
          loss, logits = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask, 
                              labels=b_labels)

          # Accumulate the training loss over all of the batches so that we can
          # calculate the average loss at the end. `loss` is a Tensor containing a
          # single value; the `.item()` function just returns the Python value 
          # from the tensor.
          total_train_loss += loss.item()

          # Perform a backward pass to calculate the gradients.
          loss.backward()

          # Clip the norm of the gradients to 1.0.
          # This is to help prevent the "exploding gradients" problem.
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # Update parameters and take a step using the computed gradient.
          # The optimizer dictates the "update rule"--how the parameters are
          # modified based on their gradients, the learning rate, etc.
          optimizer.step()

          # Update the learning rate.
          scheduler.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(train_dataloader)            
      
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)

      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epoch took: {:}".format(training_time))
          
      # ========================================
      #               Validation
      # ========================================
      # After the completion of each training epoch, measure our performance on
      # our validation set.

      print("")
      print("Running Validation...")

      t0 = time.time()

      # Put the model in evaluation mode--the dropout layers behave differently
      # during evaluation.
      model.eval()

      # Tracking variables 
      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0

      # Evaluate data for one epoch
      for batch in validation_dataloader:
          
          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using 
          # the `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)
          
          # Tell pytorch not to bother with constructing the compute graph during
          # the forward pass, since this is only needed for backprop (training).
          with torch.no_grad():        

              # Forward pass, calculate logit predictions.
              # token_type_ids is the same as the "segment ids", which 
              # differentiates sentence 1 and 2 in 2-sentence tasks.
              # The documentation for this `model` function is here: 
              # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
              # Get the "logits" output by the model. The "logits" are the output
              # values prior to applying an activation function like the softmax.
              (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
              
          # Accumulate the validation loss.
          total_eval_loss += loss.item()

          # Move logits and labels to CPU
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()

          # Calculate the accuracy for this batch of test sentences, and
          # accumulate it over all batches.
          total_eval_accuracy += flat_accuracy(logits, label_ids)
          

      # Report the final accuracy for this validation run.
      avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
      print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

      # Calculate the average loss over all of the batches.
      avg_val_loss = total_eval_loss / len(validation_dataloader)
      
      # Measure how long the validation run took.
      validation_time = format_time(time.time() - t0)
      
      print("  Validation Loss: {0:.2f}".format(avg_val_loss))
      print("  Validation took: {:}".format(validation_time))

      # Record all statistics from this epoch.
      training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Valid. Accur.': avg_val_accuracy,
              'Training Time': training_time,
              'Validation Time': validation_time
          }
      )

  print("")
  print("Training complete!")

  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
  return model, training_stats
model, training_stats = train(train_dataloader)


======== Epoch 1 / 3 ========
Training...
  Batch    40  of  1,221.    Elapsed: 0:00:33.
  Batch    80  of  1,221.    Elapsed: 0:01:06.
  Batch   120  of  1,221.    Elapsed: 0:01:39.
  Batch   160  of  1,221.    Elapsed: 0:02:12.
  Batch   200  of  1,221.    Elapsed: 0:02:45.
  Batch   240  of  1,221.    Elapsed: 0:03:19.
  Batch   280  of  1,221.    Elapsed: 0:03:52.
  Batch   320  of  1,221.    Elapsed: 0:04:25.
  Batch   360  of  1,221.    Elapsed: 0:04:58.
  Batch   400  of  1,221.    Elapsed: 0:05:31.
  Batch   440  of  1,221.    Elapsed: 0:06:04.
  Batch   480  of  1,221.    Elapsed: 0:06:37.
  Batch   520  of  1,221.    Elapsed: 0:07:10.
  Batch   560  of  1,221.    Elapsed: 0:07:43.
  Batch   600  of  1,221.    Elapsed: 0:08:17.
  Batch   640  of  1,221.    Elapsed: 0:08:50.
  Batch   680  of  1,221.    Elapsed: 0:09:23.
  Batch   720  of  1,221.    Elapsed: 0:09:56.
  Batch   760  of  1,221.    Elapsed: 0:10:29.
  Batch   800  of  1,221.    Elapsed: 0:11:02.
  Batch   840  of

In [ ]:
# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4, 5])

plt.show()

In [ ]:
def predict(dataset):

  # Create the DataLoader for  dataset.
  prediction_data = dataset
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


  # Prediction on test set

  print('Predicting labels for {:,} sentences...'.format(len(dataset)))

  # Put model in evaluation mode
  model.eval()

  # Tracking variables 
  predictions , true_labels = [], []

  # Predict 
  for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

  print('DONE.')
  return predictions, true_labels

test_predictions, test_true_labels = predict(dataset_test)
valid_predictions, valid_true_labels = predict(dataset_valid)
train_predictions, train_true_labels = predict(dataset_train)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

def metrics(predictions, true_labels):
  for i in range(len(predictions)):
    if i == 0:
      pred_flat = np.argmax(predictions[i], axis=1).flatten()
      confidence_flat = np.amax(predictions[i], axis=1).flatten()
      labels_flat = true_labels[i].flatten()
    else:
      pred_flat = np.append(pred_flat, np.argmax(predictions[i], axis=1).flatten())
      confidence_flat = np.append(confidence_flat, np.amax(predictions[i], axis=1).flatten())
      labels_flat = np.append(labels_flat, true_labels[i].flatten())

  print(len(pred_flat), len(labels_flat))
  pred_flat = label2emotion(pred_flat)
  labels_flat = label2emotion(labels_flat)
  print(classification_report(labels_flat, pred_flat))
  print(accuracy_score(labels_flat, pred_flat))
  return pred_flat, confidence_flat
pred_flat_test, confidence_flat_test = metrics(test_predictions, test_true_labels)
pred_flat_valid, confidence_flat_valid = metrics(valid_predictions, valid_true_labels)
pred_flat_train, confidence_flat_train = metrics(train_predictions, train_true_labels)

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
 
output_dir = "./%s_%s_save/" % (model_name, model_type)

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))